In [2]:
print("Hello World!")

Hello World!


In [3]:
!pip install openai python-dotenv wikipedia

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 999.0/999.0 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 52.2 MB/s eta 0:00:00
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11757 sha256=1d43586de66fc3dc8e3d0df409122f907f6e6c291d7cc897fc6da7d8ce0ee513
  Stored in directory: /home/codespace/.cache/pip/wheels/63/47/7c/a9688349aa74d228ce0a9023229c6c0ac52ca2a40fe87679b8
Successfully built wikipedia
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10/10 [openai] 9/10 [openai]c]-types]

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python3 -m pip install --upgrade pip


In [10]:
import os
import openai
import wikipedia
from dotenv import load_dotenv

load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# --- Tools ---
def calculator(expression: str) -> str:
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

def wiki_search(query: str) -> str:
    try:
        return wikipedia.summary(query, sentences=2)
    except Exception as e:
        return f"Error: {e}"

tools = {
    "calculator": calculator,
    "wiki": wiki_search,
}

# --- Agent Loop ---
def run_agent(query: str, max_steps=3):
    print(f"User: {query}\n")

    context = """You are an intelligent agent that can reason step by step.
    You have access to these tools:
    - calculator(expression): evaluate math expressions and return a number
    - wiki(query): return a short summary of the topic

    You will follow this reasoning format strictly:

    Thought: describe your reasoning
    Action: choose either calculator(), wiki(), or ANSWER()
    Observation: result of the previous action

    Important:
    - Use at most ONE tool per step.
    - If you already have the information you need, respond with:
    Action: ANSWER(<final answer here>)
    - Do NOT repeat the same tool call multiple times.
    - Stop once you have provided the final answer.
    """

    for step in range(max_steps):
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": context},
                {"role": "user", "content": f"User question: {query}\nPrevious steps:\n{context}"}
            ]
        )
        text = response.choices[0].message.content
        print(text)

        # Parse the action
        lines = text.strip().splitlines()
        action_line = [l for l in lines if l.startswith("Action:")]
        if not action_line:
            break

        action = action_line[-1].replace("Action:", "").strip()

        if "ANSWER" in action:
            final = action.split("ANSWER")[-1].strip(" :()")
            print(f"\nFinal Answer: {final}")
            return  # instead of break, to stop before next Observation print

        else:
            # Expect "tool_name(input)"
            if "(" in action and action.endswith(")"):
                tool_name = action.split("(")[0]
                tool_input = action[len(tool_name)+1:-1]

                if tool_name in tools:
                    observation = tools[tool_name](tool_input)
                else:
                    observation = f"Unknown tool: {tool_name}"

                context += f"{text}\nObservation: {observation}\n"
                print(f"Observation: {observation}\n")


In [11]:
run_agent("What is 23 * 7?")

User: What is 23 * 7?

Thought: To find the product of 23 and 7, I will use the calculator tool to evaluate the expression. 
Action: calculator(23 * 7)
Observation: 161
Observation: 161

Action: ANSWER(161)

Final Answer: 161


In [7]:
run_agent("Who is Marie Curie?")

User: Who is Marie Curie?

Thought: I need to gather information about Marie Curie to provide a summary of who she is and her contributions to science. 
Action: wiki("Marie Curie") 
Observation: Marie Curie was a Polish-born physicist and chemist who conducted pioneering research on radioactivity. She was the first woman to win a Nobel Prize, and she is the only person to win Nobel Prizes in two different scientific fields (Physics in 1903 and Chemistry in 1911). Curie's work led to the discovery of the elements polonium and radium, significantly advancing the study of radioactive materials and their applications in medicine.
Observation: Maria Salomea Skłodowska-Curie (Polish: [ˈmarja salɔˈmɛa skwɔˈdɔfska kʲiˈri] ; née Skłodowska; 7 November 1867 – 4 July 1934), known as Marie Curie ( KURE-ee; French: [maʁi kyʁi] ), was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity. 
She was the first woman to win a Nobel Prize, the first pers

In [ ]:
# --- Tools ---
def calculator(expression: str) -> str:
    try:
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

def wiki_search(query: str) -> str:
    try:
        return wikipedia.summary(query, sentences=2)
    except Exception as e:
        return f"Error: {e}"

tools = {
    "calculator": calculator,
    "wiki": wiki_search,
}

# --- Agent Function ---
def run_agent_2(query: str, max_steps: int = 3):
    print(f"User: {query}\n")

    context = """You are an intelligent agent that can reason step by step.
    You have access to these tools:
    - calculator(expression): evaluate math expressions
    - wiki(query): return a short summary of the topic

    Follow this reasoning format strictly:
    Thought: describe your reasoning
    Action: choose either calculator(), wiki(), or ANSWER()
    Observation: result of the previous action

    Rules:
    - Use at most ONE tool per step.
    - If you already know the answer, respond with:
    Action: ANSWER(<final answer here>)
    - Do NOT repeat the same tool call multiple times.
    - Stop once you have given the final answer.
    """

    for step in range(1, max_steps + 1):
        response = openai.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": context},
                {"role": "user", "content": f"User question: {query}\nPrevious reasoning:\n{context}"}
            ]
        )

        text = response.choices[0].message.content.strip()
        print(text)

        # Parse the action
        lines = text.splitlines()
        action_line = next((l for l in lines if l.startswith("Action:")), None)
        if not action_line:
            print("\nNo action detected. Stopping.")
            break

        action = action_line.replace("Action:", "").strip()

        # Check for final answer
        if "ANSWER" in action:
            final = action.split("ANSWER")[-1].strip(" :()")
            print(f"\nFinal Answer: {final}")
            return

        # Execute tool if requested
        if "(" in action and action.endswith(")"):
            tool_name = action.split("(")[0]
            tool_input = action[len(tool_name) + 1:-1]

            if tool_name in tools:
                observation = tools[tool_name](tool_input)
            else:
                observation = f"Unknown tool: {tool_name}"

            print(f"Observation: {observation}\n")

            # Update context so model remembers the new step
            context += f"\n{text}\nObservation: {observation}\n"
        else:
            print("\nCould not parse tool call. Stopping.")
            break

In [15]:
run_agent_2("What is 23 * 7?")

User: What is 23 * 7?

Thought: To find out what 23 times 7 is, I will use the calculator to compute this multiplication.
Action: calculator(23 * 7)
Observation: 161
Observation: 161

Action: ANSWER(161)

Final Answer: 161


In [16]:
run_agent_2("Who is Marie Curie?")

User: Who is Marie Curie?

Thought: I need to find information about Marie Curie, who is a notable historical figure. The best way to get concise and accurate information is to use the wiki tool. 
Action: wiki("Marie Curie")
Observation: Marie Curie was a Polish-born physicist and chemist who conducted pioneering research on radioactivity. She was the first woman to win a Nobel Prize and remains the only person to win Nobel Prizes in two different scientific fields: Physics and Chemistry.
Observation: Maria Salomea Skłodowska-Curie (Polish: [ˈmarja salɔˈmɛa skwɔˈdɔfska kʲiˈri] ; née Skłodowska; 7 November 1867 – 4 July 1934), known as Marie Curie ( KURE-ee; French: [maʁi kyʁi] ), was a Polish and naturalised-French physicist and chemist who conducted pioneering research on radioactivity. 
She was the first woman to win a Nobel Prize, the first person to win a Nobel Prize twice, and the only person to win a Nobel Prize in two scientific fields.

Action: ANSWER(Marie Curie was a Polish-b